In [1]:
import numpy as np
from collections import deque

from PIL import Image
import imageio

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.animation import ArtistAnimation
%matplotlib inline

In [2]:
clrs = ['#FEFBE9', '#FCF7D5', '#F5F3C1', '#EAF0B5', '#DDECBF',
        '#D0E7CA', '#C2E3D2', '#B5DDD8', '#A8D8DC', '#9BD2E1',
        '#8DCBE4', '#81C4E7', '#7BBCE7', '#7EB2E4', '#88A5DD',
        '#9398D2', '#9B8AC4', '#9D7DB2', '#9A709E', '#906388',
        '#805770', '#684957', '#46353A']
cmap = LinearSegmentedColormap.from_list('sron_iridescent', clrs)
cmap.set_bad('#999999')

o_cmap = np.array(cmap([1.0,7/8.,6/8.,5/8.,4/8.,3/8.,2/8.,1/8.,0.0]))[:,:3]
o_cmap = np.concatenate((np.array([[220/255.,5/255.,12/255.]]),o_cmap))
o_cmap = (o_cmap*255).astype(int)

In [3]:
def process_input(data):
    nums = []
    for line in data:
        nums.append(list(line))
    return np.array(nums).astype(int)

def step(data):
    data += 1
    flashed = {}
    
    to_flash = np.where(data > 9)
    
    for i in range(0, len(to_flash[0])):
        flashed[(to_flash[0][i],to_flash[1][i])] = True
        
    queue = deque(flashed.keys())
        
    while len(queue):
        cur_pos = queue.popleft()
        for dx in range(-1,2):
            for dy in range(-1,2):
                if dx == 0 and dy == 0:
                    continue
                nxt_pos = (cur_pos[0]+dx, cur_pos[1]+dy)
                if nxt_pos[0] < 0 or nxt_pos[1] < 0 or nxt_pos[0] > len(data)-1 or nxt_pos[1] > len(data[nxt_pos[0]]) - 1:
                    continue
                if nxt_pos not in flashed.keys():
                    data[nxt_pos[0],nxt_pos[1]] += 1
                    if data[nxt_pos[0],nxt_pos[1]] > 9:
                        queue.append(nxt_pos)
                        flashed[nxt_pos] = True
                        
    for key in flashed.keys():
        data[key[0],key[1]] = 0
        
    return data

def create_frame(data, frames, stp):
    clr_map = [[],
               [70,53,58],[],[],
               [],[],[],
               [],[],[254,251,233]]
    
    coloured = np.ones((data.shape[0],data.shape[1],3), dtype=int)
    for x in range(0, len(data)):
        for  y in range(0, len(data[x])):
            coloured[x,y] = o_cmap[data[x,y]]
    
    image = Image.fromarray(coloured.astype('uint8'), mode='RGB')
    image = image.resize((100*4,100*4), resample=Image.NEAREST)
    image.save('./frames/day11_'+str(stp).zfill(4)+'.png')
    frames.append(np.array(image))
    return frames

def create_vis(data):
    data = process_input(data)
    frames = []
    step_count = 0
    
    frames = create_frame(data, frames, step_count)
    
    while np.sum(data):
        data = step(data)
        step_count += 1
        frames = create_frame(data, frames, step_count)
        
    for i in range(1, 21):
        data = step(data)
        frames = create_frame(data, frames, step_count+i)
        
    return frames

In [4]:
inpt = np.genfromtxt('day11_input.txt', dtype=str, delimiter='\n')
frames = create_vis(inpt)

In [5]:
imageio.mimsave('./day11-vis.gif', frames, duration=1/4)